<a href="https://colab.research.google.com/github/jespimentel/sis_geo/blob/main/plotador_ddm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import folium
from folium.plugins import HeatMap, TagFilterButton

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_1 = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/registros_nos_limites_0_2000.csv')
df_2 = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/registros_nos_limites_2001_3314.csv')
df_partes = pd.read_csv('/content/drive/MyDrive/Datasets/sis_geo/partes.csv')
df = pd.concat([df_1,df_2] , ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Número TJ             3168 non-null   object 
 1   Número MP             3168 non-null   object 
 2   Tipo de Procedimento  3168 non-null   object 
 3   Situação              3168 non-null   object 
 4   Delegacia             3164 non-null   object 
 5   Assunto               3168 non-null   object 
 6   DtFatoInicial         3168 non-null   object 
 7   Violência Doméstica   3167 non-null   object 
 8   Município             3168 non-null   object 
 9   Logradouro            3168 non-null   object 
 10  Num_Logradouro        2904 non-null   object 
 11  Bairro                3019 non-null   object 
 12  Latitude              3168 non-null   float64
 13  Longitude             3168 non-null   float64
dtypes: float64(2), object(12)
memory usage: 346.6+ KB


In [4]:
df_partes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3592 entries, 0 to 3591
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Número MP  3592 non-null   object
 1   Partes     3592 non-null   object
dtypes: object(2)
memory usage: 56.2+ KB


In [5]:
# 'Delegacia de Polícia de Defesa da Mulher de Piracicaba'
criterio = (df['Delegacia'] == 'Delegacia de Polícia de Defesa da Mulher de Piracicaba') & (df['Situação'] == 'DENUNCIADO')
df_mulher = df.loc[criterio][['Número TJ', 'Assunto', 'DtFatoInicial','Latitude','Longitude']]
df_mulher['Assunto'] = df_mulher['Assunto'].apply(lambda x: x.strip())

In [6]:
# Converter a coluna 'DtFatoInicial' para o tipo datetime
df_mulher['DtFatoInicial'] = pd.to_datetime(df_mulher['DtFatoInicial'], dayfirst=False)
df_mulher['DiaDaSemana'] = df_mulher['DtFatoInicial'].dt.day_name()
df_mulher['DtFatoInicial'] = df_mulher['DtFatoInicial'].dt.strftime('%d/%m/%Y')

traducao_dias_semana = {
    'Monday': 'Segunda-feira',
    'Tuesday': 'Terça-feira',
    'Wednesday': 'Quarta-feira',
    'Thursday': 'Quinta-feira',
    'Friday': 'Sexta-feira',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}

df_mulher['DiaDaSemana'] = df_mulher['DiaDaSemana'].map(traducao_dias_semana)

# Substituição de valores
df_mulher['Assunto'] = df_mulher['Assunto'].replace('Contra a Mulher', 'Lesão Corporal')

df_mulher

,Número TJ,Assunto,DtFatoInicial,Latitude,Longitude,DiaDaSemana
15,1500777-54.2023.8.26.0599,Vias de fato,09/04/2023,-22.77327,-47.59884,Domingo
16,1500772-32.2023.8.26.0599,Lesão Corporal,07/04/2023,-22.74380,-47.64222,Sexta-feira
29,1500755-93.2023.8.26.0599,Ameaça,04/04/2023,-22.76253,-47.64963,Terça-feira
30,1500754-11.2023.8.26.0599,Ameaça,03/04/2023,-22.74251,-47.62412,Segunda-feira
39,1500739-42.2023.8.26.0599,Ameaça,01/04/2023,-22.74186,-47.66518,Sábado
...,...,...,...,...,...,...
3120,1501529-19.2022.8.26.0451,Injúria,06/01/2022,-22.68929,-47.65556,Quinta-feira
3131,1500986-16.2022.8.26.0451,Ameaça,05/01/2022,-22.74730,-47.65632,Quarta-feira
3132,1500028-71.2022.8.26.0599,Vias de fato,04/01/2022,-22.76580,-47.63525,Terça-feira
3134,1500024-34.2022.8.26.0599,DESOBEDIÊNCIA À MEDIDA PROTETIVA DE URGÊNCIA D...,04/01/2022,-22.68149,-47.65211,Terça-feira


In [7]:
assuntos = list(set(df_mulher['Assunto'].tolist()))
dias_da_semana = list(set(df_mulher['DiaDaSemana'].tolist()))
assuntos

['Injúria',
 'Lesão Corporal',
 'Estupro',
 'DESOBEDIÊNCIA À\xa0MEDIDA PROTETIVA DE URGÊNCIA DA LEI MARIA DA PENHA',
 'Violação de domicílio',
 'Importunação Sexual',
 'Lesão Cometida em Razão da Condição de Mulher',
 'Resistência',
 'Perseguição',
 'Ameaça',
 'Crime de Descumprimento de Medida Protetiva  de Urgência',
 'Violência Psicológica contra a Mulher',
 'Dano',
 'Feminicídio',
 'Contra pessoas não identificadas como mulher',
 'Vias de fato',
 'Estupro de vulnerável',
 'Calúnia',
 'Crime Tentado',
 'Violação sexual mediante fraude']

In [8]:
import folium
import random

# Dicionário para mapear cores aos assuntos
cores_assuntos = {}
cores_predefinidas = ['darkgreen', 'pink', 'lightgreen', 'darkpurple', 'lightgray', 'gray',
                      'beige', 'red', 'lightred', 'blue', 'cadetblue', 'darkred', 'black',
                      'purple', 'lightblue', 'white', 'green', 'darkblue', 'orange']
for assunto in assuntos:
    cor_assunto = random.choice(cores_predefinidas)
    cores_assuntos[assunto] = cor_assunto

PIRA_COORDINATES = (-22.769960, -47.647579)

# Cria um mapa vazio em Piracicaba
_map = folium.Map(location=PIRA_COORDINATES, zoom_start=12)

# Adiciona um título ao mapa
title_html = """
<h1 style="text-align:center;"><b>DDM - Casos denunciados para data do fato entre 01/01/2022 e 24/04/2023</b></h1>
<h2 style="text-align:center;"><b>Base: SISMP INTEGRADO - Extração em 24/04/2023</b></h2>
"""
_map.get_root().html.add_child(folium.Element(title_html))

# Adiciona o marcador para cada ocorrência
for each in df_mulher.iterrows():
    assunto = each[1]['Assunto']
    dia_da_semana = each[1]['DiaDaSemana']
    cor_assunto = cores_assuntos[assunto]
    folium.Marker(
        location=[each[1]["Latitude"], each[1]["Longitude"]],
        clustered_marker=True,
        tags=[assunto, dia_da_semana],
        popup=f"{assunto}<br>{each[1]['DtFatoInicial']}<br>{each[1]['Número TJ']}<br>{each[1]['DiaDaSemana']}",
        icon=folium.Icon(color=cor_assunto)
    ).add_to(_map)

# Adiciona botões de filtro por tag
TagFilterButton(assuntos, clear_text='Todos', name='Assuntos').add_to(_map)
TagFilterButton(dias_da_semana, clear_text="Todos",name='DiaDaSemana').add_to(_map)

display(_map)
_map.save('violencia_domestica.html')

In [9]:
# Mapa de calor

# Extrai as colunas de latitude e longitude do dataset
latitude = df_mulher['Latitude']
longitude = df_mulher['Longitude']

PIRA_COORDINATES = (-22.769960, -47.647579)

# Cria um mapa vazio em Piracicaba
mapa = folium.Map(location=PIRA_COORDINATES, zoom_start=12)

# Agrupa as coordenadas de latitude e longitude em uma lista
dados = list(zip(latitude, longitude))

# Adiciona o mapa de calor
heatmap = HeatMap(dados).add_to(mapa)

# Adiciona um título ao mapa
title_html = """
<h1 style="text-align:center;"><b>Mapa de calor da violência doméstica - denunciados para data do fato entre 01/01/2022 e 24/04/2023</b></h1>
<h2 style="text-align:center;"><b>Base: SISMP INTEGRADO - Extração em 24/04/2023</b></h2>
"""
mapa.get_root().html.add_child(folium.Element(title_html))

folium.TileLayer('openstreetmap').add_to(mapa)
folium.LayerControl().add_to(mapa)

# Salva o mapa em um arquivo HTML
mapa.save("heatmap_violencia_domestica.html")
mapa